# Data Aggregation & Integration

This notebook calculates the mean and median number of power outages in the 2016 Eagle-I dataset and merges the aggregated results with the demographic data in the 2010 Census dataset (CtyAvDemog2010.csv).

For more information about merging dataframes, refer to:
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

### Aggregating time series data to get mean and median

Now that we’ve learned how to work with and visualize time series data, it would also be useful to know how to aggregate data across a time series. 

In [1]:
!pip install pandas

In [2]:
import pandas as pd

In [3]:
# Load up Eagle-I 2016 Dataset
import pandas as pd
ei_2016_df = pd.read_csv("data/eaglei_outages_2016.csv", delimiter=',', skiprows=0, low_memory=False)

In [4]:
# Converting the run_start_time column to pandas' 'Timestamp' format
ei_2016_df["datetimeformat"] = pd.to_datetime(ei_2016_df["run_start_time"], format="%Y-%m-%d %H:%M:%S")
ei_2016_df


,fips_code,county,state,sum,run_start_time,datetimeformat
0,2122,Kenai Peninsula,Alaska,657,2016-01-01 00:00:00,2016-01-01
1,5003,Ashley,Arkansas,7,2016-01-01 00:00:00,2016-01-01
2,5029,Conway,Arkansas,2,2016-01-01 00:00:00,2016-01-01
3,5069,Jefferson,Arkansas,41,2016-01-01 00:00:00,2016-01-01
4,5081,Little River,Arkansas,61,2016-01-01 00:00:00,2016-01-01
...,...,...,...,...,...,...
13306019,54061,Monongalia,West Virginia,472,2016-12-31 00:00:00,2016-12-31
13306020,54069,Ohio,West Virginia,37,2016-12-31 00:00:00,2016-12-31
13306021,54075,Pocahontas,West Virginia,200,2016-12-31 00:00:00,2016-12-31
13306022,54099,Wayne,West Virginia,2,2016-12-31 00:00:00,2016-12-31


In [5]:
# Selecting only data for June 20
ei_20160620_df = ei_2016_df.loc[(ei_2016_df["datetimeformat"] >= "2016-06-19") 
                                & (ei_2016_df["datetimeformat"] < "2016-06-20")]


Say you want to find the average of power outages across the whole day for Los Angeles county for June 20. But we know that the data is in 15 minute chunks, and that the run_start_time column is in UTC so has to be adjusted to get June 20 local time.

In [6]:
# to get only Los Angeles, let's select by the FIPS code and also make sure that run_start_time is
# within 2016-06-20 00:00:00 and 2016-06-21 00:00:00 Pacific Standard Time (or in the case of June
# Pacific Daylight Time, which is 7 hours behind UTC)
la_outages_0620_df = ei_2016_df.loc[(ei_2016_df["fips_code"]==6037) 
                                          & (ei_2016_df["run_start_time"]>="2016-06-19 17:00:00")
                                          & (ei_2016_df["run_start_time"]<="2016-06-20 17:00:00")]
la_outages_0620_df


,fips_code,county,state,sum,run_start_time,datetimeformat
5953439,6037,Los Angeles,California,167,2016-06-19 17:00:00,2016-06-19 17:00:00
5953892,6037,Los Angeles,California,144,2016-06-19 17:15:00,2016-06-19 17:15:00
5954341,6037,Los Angeles,California,152,2016-06-19 17:30:00,2016-06-19 17:30:00
5954792,6037,Los Angeles,California,153,2016-06-19 17:45:00,2016-06-19 17:45:00
5955241,6037,Los Angeles,California,162,2016-06-19 18:00:00,2016-06-19 18:00:00
...,...,...,...,...,...,...
5993451,6037,Los Angeles,California,3404,2016-06-20 16:00:00,2016-06-20 16:00:00
5994131,6037,Los Angeles,California,3433,2016-06-20 16:15:00,2016-06-20 16:15:00
5994808,6037,Los Angeles,California,3706,2016-06-20 16:30:00,2016-06-20 16:30:00
5995490,6037,Los Angeles,California,3498,2016-06-20 16:45:00,2016-06-20 16:45:00


We then need to use the groupby method to group the ei_outages_0620_df by the fips code. 
This is necessary since pandas needs to group the data first before it can do any aggregation operations.

In [7]:
# Grouping by the fips code so that we can aggregate the whole day's data on a county by county basis
# (each fips code represents a county)
ei_fipscode_group = ei_20160620_df.groupby("fips_code")


In [8]:
# Now we can aggregate the data in each group (i.e. in each county) to get the mean and median of the  (and 
# others too if you want) 
ei_fipscode_agg = ei_fipscode_group.agg({"sum":["mean", "median"] })
ei_fipscode_agg

sum       
                 mean median
fips_code                   
1003        23.458333   15.0
1031         5.060606    1.0
1039         1.000000    1.0
1041         2.238095    2.0
1089        10.577778    6.0
...               ...    ...
55129        6.789474    2.0
55131        1.000000    1.0
55133       30.552239    3.0
55137        1.000000    1.0
56021      397.230769   80.0

[1399 rows x 2 columns]

Notice that there are multiple levels of indexes in the dataframe 
- In order to merge the demographics data with the mean and median power outages, we need to convert the dataframe to a form so that the 2 datasets can be easily merged

In [9]:
# Check level 1 - Index(['mean', 'median'], dtype='object')
level1 = ei_fipscode_agg.columns.get_level_values(1) # mean and median
level1

Index(['mean', 'median'], dtype='object')

In [10]:
# Assign column names to level 1 (mean and median)
ei_fipscode_agg.columns = level1 

In [11]:
# View aggregated dataset
ei_fipscode_agg

,mean,median
fips_code,,
1003,23.458333,15.0
1031,5.060606,1.0
1039,1.000000,1.0
1041,2.238095,2.0
1089,10.577778,6.0
...,...,...
55129,6.789474,2.0
55131,1.000000,1.0
55133,30.552239,3.0


In [12]:
# Reset the index
county_agg = ei_fipscode_agg.reset_index()

# View aggregated dataset
county_agg

,fips_code,mean,median
0,1003,23.458333,15.0
1,1031,5.060606,1.0
2,1039,1.000000,1.0
3,1041,2.238095,2.0
4,1089,10.577778,6.0
...,...,...,...
1394,55129,6.789474,2.0
1395,55131,1.000000,1.0
1396,55133,30.552239,3.0
1397,55137,1.000000,1.0


Now, we have a dataset (CtyAvDemog2010.csv) that contains information from the 2010 census with socioeconomic measures, including energy burden, health, and age demographics, averaged by county. 

What if we wanted to compare the average number of power outages with the average socio-economic measures? 

The first step would be to identify: 
- What do these two datasets have in common? Do they have a column/variable that exist in both datasets?


In [13]:
# Load up Demographic Dataset
df = pd.read_csv("data/CtyAvDemog2010.csv",delimiter=',', skiprows=0, low_memory=False, encoding='latin-1')
# To fix error about invalid continuation byte, fix encoding: https://stackoverflow.com/questions/5552555/unicodedecodeerror-invalid-continuation-byte

In [14]:
# View Demographic Data
df

,OBJECTID,STATEFP,COUNTYFP,GEOID,NAME,Lat,Lon,Avg_PercentBlackorAfricanAmericanalone,Avg_PercentAmericanIndian_AlaskaNative,Avg_PercentAsian,...,Avg_UnemploymentPct,Avg_PercentOfIndividualsBelow200PctFederalPovertyLinePctile,Avg_PercentOfIndividualsBelow200PctFederalPovertyLine,Avg_PercentOfIndividualsLt10oPctFederalPovertyLinePctile,Avg_PercentOfIndividualsLt100PctFederalPovertyLine,Avg_PercentIndividualsAge25OrOverWithLessThanHighSchoolDegreePct,Avg_PercentIndividualsAge25OrOverWithLessThanHighSchoolDegree,Avg_PercentOfResidentsWhoAreNotCurrentlyEnrolledInHigherEd,Avg_UnemploymentPctIn2009IslandAnd2010StatesAndPR,Avg_PercentageHouseholdsBelow100PctOfFederalPovertyLineIn2009Isl
0,1,31,39,31039,Cuming,41.915865,-96.788517,0.000000,0.000000,0.000000,...,2.333333,43.666667,27.000000,29.000000,6.666667,49.000000,10.333333,95.333333,1.333333,11.333333
1,2,53,69,53069,Wahkiakum,46.294638,-123.424458,0.010000,0.000000,0.010000,...,7.000000,39.000000,25.000000,30.000000,7.000000,40.000000,7.000000,96.000000,7.000000,12.000000
2,3,35,11,35011,De Baca,34.359273,-104.368696,0.010000,0.000000,0.000000,...,10.000000,81.000000,50.000000,64.000000,16.000000,70.000000,15.000000,96.000000,0.000000,21.000000
3,4,31,109,31109,Lancaster,40.783547,-96.688658,0.040811,0.005000,0.040541,...,2.816901,45.732394,31.112676,46.774648,13.929577,30.729730,6.662162,84.608108,4.041667,15.876712
4,5,31,129,31129,Nuckolls,40.176492,-98.046842,0.000000,0.000000,0.015000,...,1.000000,60.000000,35.500000,45.500000,10.500000,39.500000,7.500000,96.500000,2.500000,17.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,3229,13,123,13123,Gilmer,34.690506,-84.454624,0.004000,0.008000,0.002000,...,3.800000,61.400000,36.600000,62.200000,15.800000,73.400000,17.200000,96.400000,6.000000,16.400000
3229,3230,27,135,27135,Roseau,48.761068,-95.821504,0.010000,0.010000,0.018000,...,2.200000,38.600000,24.600000,34.200000,7.800000,34.400000,6.400000,96.400000,3.000000,9.600000
3230,3231,28,89,28089,Madison,32.634370,-90.034160,0.394762,0.000476,0.026667,...,4.857143,38.714286,26.476190,34.666667,9.761905,38.428571,9.000000,91.238095,4.619048,13.857143
3231,3232,48,227,48227,Howard,32.303471,-101.438772,0.045000,0.008000,0.002000,...,6.500000,59.800000,37.700000,59.600000,18.100000,73.600000,19.100000,95.400000,5.700000,17.700000


Upon further inspection, there is a geographical variable shared between the power outage (eagle-i) and demographics data, identified by the fips_code and GEOID columns respectively.

In [15]:
# Rename fips_code as GEOID
agg2 = county_agg.rename(columns={"fips_code": "GEOID"})

In [16]:
# Merge on GEOID to get mean & median number of power outages with the Demographics data
all_df = pd.merge(df, agg2, on = 'GEOID', how = 'left')

In [17]:
# View combined dataframe with census data, mean & median power outages 
all_df.sort_values("GEOID")

,OBJECTID,STATEFP,COUNTYFP,GEOID,NAME,Lat,Lon,Avg_PercentBlackorAfricanAmericanalone,Avg_PercentAmericanIndian_AlaskaNative,Avg_PercentAsian,...,Avg_PercentOfIndividualsBelow200PctFederalPovertyLine,Avg_PercentOfIndividualsLt10oPctFederalPovertyLinePctile,Avg_PercentOfIndividualsLt100PctFederalPovertyLine,Avg_PercentIndividualsAge25OrOverWithLessThanHighSchoolDegreePct,Avg_PercentIndividualsAge25OrOverWithLessThanHighSchoolDegree,Avg_PercentOfResidentsWhoAreNotCurrentlyEnrolledInHigherEd,Avg_UnemploymentPctIn2009IslandAnd2010StatesAndPR,Avg_PercentageHouseholdsBelow100PctOfFederalPovertyLineIn2009Isl,mean,median
1328,1329,1,1,1001,Autauga,32.532237,-86.646439,0.215000,0.000833,0.002500,...,33.166667,60.500000,16.166667,58.250000,12.416667,93.416667,5.000000,11.166667,NaN,NaN
1246,1247,1,3,1003,Baldwin,30.659218,-87.746067,0.090968,0.005484,0.005806,...,29.096774,44.322581,11.000000,43.290323,9.419355,95.354839,4.709677,11.322581,23.458333,15.0
1827,1828,1,5,1005,Barbour,31.870253,-85.405103,0.477778,0.001111,0.001111,...,54.333333,87.666667,30.222222,85.444444,25.666667,95.555556,6.222222,24.777778,NaN,NaN
3014,3015,1,7,1007,Bibb,33.015893,-87.127148,0.170000,0.000000,0.000000,...,41.000000,65.750000,18.500000,72.000000,19.750000,96.750000,7.000000,11.500000,NaN,NaN
855,856,1,9,1009,Blount,33.977357,-86.566440,0.010000,0.000000,0.000000,...,36.111111,55.111111,13.222222,77.000000,19.888889,95.222222,6.333333,13.111111,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,261,72,151,72151,Yabucoa,18.059858,-65.859871,0.770000,0.000000,0.000000,...,80.375000,98.000000,53.000000,89.375000,28.250000,91.250000,17.500000,51.250000,NaN,NaN
968,969,72,153,72153,Yauco,18.085669,-66.857901,0.035455,0.001818,0.000000,...,83.636364,96.454545,51.090909,83.272727,26.909091,90.727273,25.090909,55.727273,NaN,NaN
1248,1249,78,10,78010,St. Croix,17.735321,-64.746741,0.723333,0.000000,0.005333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,25.466667,NaN,NaN
1460,1461,78,20,78020,St. John,18.330435,-64.735261,0.535000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.500000,14.500000,NaN,NaN


In [18]:
# View number of non-empty records in each column
all_df.count(axis = 0)

OBJECTID                                                            3233
STATEFP                                                             3233
COUNTYFP                                                            3233
GEOID                                                               3233
NAME                                                                3233
                                                                    ... 
Avg_PercentOfResidentsWhoAreNotCurrentlyEnrolledInHigherEd          3233
Avg_UnemploymentPctIn2009IslandAnd2010StatesAndPR                   3233
Avg_PercentageHouseholdsBelow100PctOfFederalPovertyLineIn2009Isl    3233
mean                                                                1399
median                                                              1399
Length: 89, dtype: int64

In [19]:
# View data sorted by mean
all_df.sort_values("mean")

,OBJECTID,STATEFP,COUNTYFP,GEOID,NAME,Lat,Lon,Avg_PercentBlackorAfricanAmericanalone,Avg_PercentAmericanIndian_AlaskaNative,Avg_PercentAsian,...,Avg_PercentOfIndividualsBelow200PctFederalPovertyLine,Avg_PercentOfIndividualsLt10oPctFederalPovertyLinePctile,Avg_PercentOfIndividualsLt100PctFederalPovertyLine,Avg_PercentIndividualsAge25OrOverWithLessThanHighSchoolDegreePct,Avg_PercentIndividualsAge25OrOverWithLessThanHighSchoolDegree,Avg_PercentOfResidentsWhoAreNotCurrentlyEnrolledInHigherEd,Avg_UnemploymentPctIn2009IslandAnd2010StatesAndPR,Avg_PercentageHouseholdsBelow100PctOfFederalPovertyLineIn2009Isl,mean,median
1973,1974,13,299,13299,Ware,31.050881,-82.421507,0.350000,0.003333,0.011111,...,54.666667,76.666667,25.777778,72.222222,18.333333,95.444444,6.222222,21.666667,1.0,1.0
2301,2302,31,19,31019,Buffalo,40.855226,-99.074983,0.006364,0.000000,0.011818,...,33.181818,49.454545,14.181818,34.545455,6.636364,86.363636,2.000000,13.636364,1.0,1.0
973,974,55,89,55089,Ozaukee,43.360752,-87.499304,0.008333,0.000556,0.017222,...,14.611111,19.944444,4.944444,13.666667,2.722222,91.833333,3.000000,4.277778,1.0,1.0
518,519,36,41,36041,Hamilton,43.657879,-74.502456,0.007500,0.000000,0.000000,...,25.000000,39.000000,8.750000,70.750000,20.250000,96.250000,1.250000,9.250000,1.0,1.0
2310,2311,37,197,37197,Yadkin,36.158765,-80.665164,0.031429,0.002857,0.000000,...,37.428571,59.571429,15.000000,74.857143,17.571429,95.571429,4.571429,13.428571,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3225,3226,40,47,40047,Garfield,36.378273,-97.787729,0.019167,0.015833,0.007500,...,33.750000,46.083333,12.083333,50.000000,10.833333,93.750000,2.583333,15.333333,NaN,NaN
3226,3227,53,59,53059,Skamania,46.024785,-121.953232,0.002000,0.012000,0.002000,...,25.200000,54.800000,12.800000,42.400000,8.200000,96.000000,4.400000,7.800000,NaN,NaN
3228,3229,13,123,13123,Gilmer,34.690506,-84.454624,0.004000,0.008000,0.002000,...,36.600000,62.200000,15.800000,73.400000,17.200000,96.400000,6.000000,16.400000,NaN,NaN
3229,3230,27,135,27135,Roseau,48.761068,-95.821504,0.010000,0.010000,0.018000,...,24.600000,34.200000,7.800000,34.400000,6.400000,96.400000,3.000000,9.600000,NaN,NaN


A few things to consider:

1. Many of these counties have very different populations so the mean number of outages per time calculated for a county with a small number of customers may represent a larger fraction of county’s population being without power than a similar mean for a county with a large population though each of those similar sized means represent a similar number of customers without power. 
In order to get a baseline, you could measure a given county's average power outage against the total number of power customers in that county.

2. If you use this to get the mean number of outages for all counties in the US, note that Eagle-I in 2016 did not have data for all U.S. counties. 